In [1]:
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
import string
import random
#nltk.download("punkt")
#nltk.download("wordnet")

filename = "Chatbot_Train.csv"
num_inputs = 5

data = pd.read_csv(filename)
data

,input_1,input_2,input_3,input_4,input_5,topic
0,What is my altitude?,How high up am I?,Altitude here,Height,Alt,altitude
1,What is the oxygen content?,Can I breath here?,Oxygen,Is there air on this planet?,Air,oxygen
2,How fast is the wind?,What is the airflow like?,What is the speed of the air on this planet?,Wind,Airflow,airflow
3,What is the atmosphere like?,Atmosphere contents,What is in the water?,Atmosphere,What is in the air here?,atmosphere
4,What is the gravity on this planet?,Gravity,What is the force pulling me down?,NaN,How fast fo I fall on this planet?,gravity
5,Humidity,What is the humidity here?,How moist is the air on this planet?,What is the water content in the air?,How humid is it here?,humidity
6,Magnetic field,What is the magnetic force on this planet?,What is the force that keeps the planet together?,NaN,NaN,magnetic field
7,Pressure,What is the pressure on the planet?,What is the force exerted on my body here?,How much is being pushed on me on this planet?,Would I get crushed here?,pressure
8,Radiation,What is the amount of radiation on this planet?,How much energy do I get on this planet?,Can I get sick here?,Wave energy,radiation
9,What is the year length here?,How long is a year on this planet?,How old would I be on this planet?,Year length,How long does it take for this planet to orbit...,year


In [2]:
# Lemmatizing wordss
lemmatizer = WordNetLemmatizer()
words = []
topics = []

X = []
y = []
inputs = []

# Loop csv append topic
for i, j in data.iterrows():
    class_value = data.iloc[i]["topic"]
    if not pd.isna(class_value):
        for k in range(num_inputs):
            col_value = k+1
            row_input = data.iloc[i]["input_"+str(col_value)]
            if not pd.isna(row_input):
                tokens = nltk.word_tokenize(row_input)
                words.extend(tokens)
                X.append(row_input)
                y.append(class_value)
    # Add class if topic not present in list
    if class_value not in topics:
        topics.append(class_value)
# Lemmatize all the words in the vocab and convert them to lowercase if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
topics = sorted(set(topics))

print(words)
print(topics)
print(X)
print(y)

["'s", 'a', 'air', 'airflow', 'alt', 'altitude', 'am', 'amount', 'atmosphere', 'be', 'being', 'body', 'breath', 'can', 'content', 'crushed', 'do', 'doe', 'down', 'energy', 'exerted', 'fall', 'fast', 'field', 'fo', 'for', 'force', 'get', 'gravity', 'height', 'here', 'high', 'how', 'humid', 'humidity', 'i', 'in', 'is', 'it', 'keep', 'know', 'length', 'like', 'location', 'long', 'magnetic', 'me', 'moist', 'much', 'my', 'name', 'of', 'old', 'on', 'orbit', 'oxygen', 'planet', 'player', 'pressure', 'pulling', 'pushed', 'radiation', 'sick', 'speed', 'sun', 'take', 'that', 'the', 'their', 'there', 'this', 'to', 'together', 'up', 'water', 'wave', 'what', 'where', 'who', 'wind', 'would', 'year', 'you']
['airflow', 'altitude', 'atmosphere', 'gravity', 'humidity', 'magnetic field', 'oxygen', 'planet', 'player', 'pressure', 'radiation', 'year']
['What is my altitude?', 'How high up am I?', 'Altitude here', 'Height', 'Alt', 'What is the oxygen content?', 'Can I breath here?', 'Oxygen', 'Is there air

In [3]:
training = []
out_empty = [0] * len(topics)
for idx, doc in enumerate(X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row = list(out_empty)
    output_row[topics.index(y[idx])] = 1
    training.append([bow, output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [4]:
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200
# Deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               10752     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 12)                780       
                                                                 
Total params: 19,788
Trainable params: 19,788
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
2/2 [==============================

2/2 [==============================] - 0s 4ms/step - loss: 0.0394 - accuracy: 1.0000
Epoch 74/200
2/2 [==============================] - 0s 4ms/step - loss: 0.0350 - accuracy: 0.9825
Epoch 75/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0141 - accuracy: 1.0000
Epoch 76/200
2/2 [==============================] - 0s 4ms/step - loss: 0.0150 - accuracy: 1.0000
Epoch 77/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0222 - accuracy: 0.9825
Epoch 78/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 79/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0105 - accuracy: 1.0000
Epoch 80/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0150 - accuracy: 1.0000
Epoch 81/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 82/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 83/200
2/2 [===============

2/2 [==============================] - 0s 3ms/step - loss: 0.0435 - accuracy: 0.9825
Epoch 157/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0118 - accuracy: 1.0000
Epoch 158/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 159/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0136 - accuracy: 1.0000
Epoch 160/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0381 - accuracy: 0.9825
Epoch 161/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 162/200
2/2 [==============================] - 0s 2ms/step - loss: 0.0126 - accuracy: 1.0000
Epoch 163/200
2/2 [==============================] - 0s 2ms/step - loss: 0.0194 - accuracy: 0.9825
Epoch 164/200
2/2 [==============================] - 0s 4ms/step - loss: 0.0267 - accuracy: 0.9825
Epoch 165/200
2/2 [==============================] - 0s 3ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 166/200
2/2 [=====

In [5]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

In [8]:
# running the chatbot
while True:
    message = input("")
    intents = pred_class(message, words, topics)
    print(intents)

What planet am I on currently?
['planet']
Height
['oxygen', 'pressure']
How fast is the wind here?
['airflow']
Super windy
['oxygen', 'pressure']
The wind is super fast on this planet
['airflow']


KeyboardInterrupt: 